## CALCOLO OPINION LEADER

In [1]:
import pandas as pd
import kmeans

In [4]:
tweet_users = pd.read_csv("../datasets/twitter/twitter_users.csv")
tweet_influenti = pd.read_csv("../dataframes/filtered_tweets_with_collections.csv")
tweets_sentiment = pd.read_csv("../dataframes/tweets_sentiment_df.csv")